In [1]:
import csv

from matplotlib.pyplot import *
from numpy import *



# Funcion para convertir un archivo tipo .csv a una lista.
# Usado para la entrada de entrenamiento y test
def convertir_csv(filename):
    data = []
    file = open(filename, newline='')
    reader = csv.reader(file, delimiter=',')
    for row in reader:
        entrada1 = float(row[0])
        entrada2 = float(row[1])
        entrada3 = float(row[2])
        entrada4 = float(row[3])
        data.append([entrada1, entrada2, entrada3, entrada4])
    return data


# Funcion para convertir un archivo tipo .csv a una lista.
# Usado para la salida de entrenamiento
def convertir_scsv(filename):
    data = []
    file = open(filename, newline='')
    reader = csv.reader(file, delimiter=',')
    for row in reader:
        # row = [Salida 1 (int)]
        salida1 = float(row[0])
        data.append([salida1])
    return data


error_capa1 = []
error_capa2 = []
error_capa3 = []
error_capa4 = []
error_capa5 = []


# Funcion para calcular el porcentaje de aciertos.
def calculatesucc(y_test_obtenido):
    y_test_deseado = array(convertir_scsv('./Y_test_deseado.csv'))    
    acum0 = 0
    for i in range(len(y_test_obtenido)):
        if abs(y_test_deseado[i] - y_test_obtenido[i]) == 0:
            acum0 += 1
    print("Porcentaje de aciertos")
    print(str(round((acum0 / len(y_test_obtenido)) * 100, 2)) + '%')


# Funcion para graficar el error.
def ploterror():
    subplot(231)
    plot(error_capa1)
    xlabel('Iteración')
    ylabel('Error')
    title("Capa 1")

    subplot(232)
    plot(error_capa2)
    xlabel('Iteración')
    ylabel('Error')
    title("Capa 2")

    subplot(233)
    plot(error_capa3)
    xlabel('Iteración')
    ylabel('Error')
    title("Capa 3")

    subplot(234)
    plot(error_capa4)
    xlabel('Iteración')
    ylabel('Error')
    title("Capa 4")

    subplot(235)
    plot(error_capa5)
    xlabel('Iteración')
    ylabel('Error')
    title("Capa 5")
    suptitle('Evolución del error')
    show()


# Funcion para convertir una lista en un archivo .csv
def tocsv(filename, data):
    file = open(filename, 'w', newline='')
    writer = csv.writer(file)
    for i in range(len(data)):
        writer.writerow(data[i])


class CapaNeuronal:
    def __init__(self, cantidad_de_neuronas, cantidad_de_entradas_por_neurona):
        self.pesos = 2 * random.random((cantidad_de_entradas_por_neurona, cantidad_de_neuronas)) - 1


class RedNeuronal:
    def __init__(self, capa1, capa2, capa3, capa4, capa5):
        self.capa1 = capa1
        self.capa2 = capa2
        self.capa3 = capa3
        self.capa4 = capa4
        self.capa5 = capa5

    # La función sigmoidal, que describe una curva en forma de S
    # Se pasan la suma ponderada de las entradas a través de esta función para 
    # normalizarlos entre 0 y 1.
    def __sigmoidal(self, x):
        return 1 / (1 + exp(-x))

    # La derivada de la función sigmoidal.
    # Este es el gradiente de la función sigmoidal.
    # Indica la confianza que tenemos en el peso existente.
    def __derivada_sigmoidal(self, x):
        return x * (1 - x)

    # Entrenamos a la red neuronal a través de un proceso de prueba y error
    # Se realiza un ajuste de los pesos sinápticos cada vez.
    def entrenar(self, entrenamiento_set_entradas, entrenamiento_set_salida, cantidad_de_ciclos, tasa_aprendizaje):
        for iteration in range(cantidad_de_ciclos): 
            salida_de_capa_1, salida_de_capa_2, salida_de_capa_3, salida_de_capa_4, salida_de_capa_5 = self.think(entrenamiento_set_entradas)            
      
            # Calcula el error de la capa 5
            capa5_error = entrenamiento_set_salida - salida_de_capa_5
            capa5_delta = capa5_error * self.__derivada_sigmoidal(salida_de_capa_5)

            # Calcula el error de la capa 4
            capa4_error = capa5_delta.dot(self.capa5.pesos.T)
            capa4_delta = capa4_error * self.__derivada_sigmoidal(salida_de_capa_4)

            # Calcula el error de la capa 3
            capa3_error = capa4_delta.dot(self.capa4.pesos.T)
            capa3_delta = capa3_error * self.__derivada_sigmoidal(salida_de_capa_3)

            # Calcula el error de la capa 2(La diferencia entre el resultado deseado
            # y el resultado obtenido).
            capa2_error = capa3_delta.dot(self.capa3.pesos.T)
            capa2_delta = capa2_error * self.__derivada_sigmoidal(salida_de_capa_2)

            # Calcula el error de la capa 1 (Inspeccionando los pesos en la capa 1,
            # podemos determinar cuanto influye en el error de la capa 2).
            capa1_error = capa2_delta.dot(self.capa2.pesos.T)
            capa1_delta = capa1_error * self.__derivada_sigmoidal(salida_de_capa_1)

            # Se calcula el ajuste de pesos
            self.capa1.pesos += tasa_aprendizaje * entrenamiento_set_entradas.T.dot(capa1_delta)
            self.capa2.pesos += tasa_aprendizaje * salida_de_capa_1.T.dot(capa2_delta)
            self.capa3.pesos += tasa_aprendizaje * salida_de_capa_2.T.dot(capa3_delta)
            self.capa4.pesos += tasa_aprendizaje * salida_de_capa_4.T.dot(capa4_delta)
            self.capa5.pesos += tasa_aprendizaje * salida_de_capa_5.T.dot(capa5_delta)

            error_capa1.append(capa1_error[0])
            error_capa2.append(capa2_error[0])
            error_capa3.append(capa3_error[0])
            error_capa4.append(capa4_error[0])
            error_capa5.append(capa5_error[0])
            

    # La red neuronal realiza el producto punto en base a las entradas
    def think(self, entradas):
        salida_de_capa1 = self.__sigmoidal(dot(entradas, self.capa1.pesos))
        salida_de_capa2 = self.__sigmoidal(dot(salida_de_capa1, self.capa2.pesos))
        salida_de_capa3 = self.__sigmoidal(dot(salida_de_capa2, self.capa3.pesos))
        salida_de_capa4 = self.__sigmoidal(dot(salida_de_capa3, self.capa4.pesos))
        salida_de_capa5 = self.__sigmoidal(dot(salida_de_capa4, self.capa5.pesos))
        
        return salida_de_capa1, salida_de_capa2, salida_de_capa3, salida_de_capa4, salida_de_capa5

    # Funcion para imprimir los pesos
    def print_weights(self):
        print("    Capa 1 (4 neuronas con 4 entradas): ")
        print(self.capa1.pesos)
        print("    Capa 2 (1 neurona con 4 entradas):")
        print(self.capa2.pesos)
        print("    Capa 3 (1 neurona con 4 entradas):")
        print(self.capa3.pesos)


if __name__ == "__main__":
    # Definimos una semilla del generador de números aleatorios, a fin de generar
    # los mismo números aleatorios en cada proceso.
    random.seed(1)

    # Creamos la capa 1 (4 neuronas con 4 entradas)
    capa1 = CapaNeuronal(6, 4)

    # Creamos la capa 2 (10 neuronas con 4 entradas)
    capa2 = CapaNeuronal(6, 6)

    # Creamos la capa 3 (4 neurona con 10 entradas)
    capa3 = CapaNeuronal(6, 6)
    
        # Creamos la capa 4 (4 neurona con 10 entradas)
    capa4 = CapaNeuronal(6, 6)

    # Creamos la capa 5 (1 neurona con 10 entradas)
    capa5 = CapaNeuronal(1, 6)

        # Combinamos las capas para crear la red neuronal
    red_neuronal = RedNeuronal(capa1, capa2, capa3, capa4, capa5)

    # Entrenar la red neuronal utilizando el conjunto de entrenamiento.
    print("Entrenando")
    # e_s_e : entrenamiento_set_entradas
    e_s_e = array(convertir_csv('./X_train.csv'))
    # e_s_s : entrenamiento_set_salida
    e_s_s = array(convertir_scsv('./Y_train.csv'))
    # t_a : tasa_aprendizaje
    t_a = 0.0005
    ciclos = 3000
    red_neuronal.entrenar(e_s_e, e_s_s, ciclos, t_a)
    print("Fin entrenamiento")

    # Probamos la red neuronal con datos de prueba.
    print("Probamos la red neuronal con datos de prueba: ")
    # test: set_pruebas
    x_test = array(convertir_csv('./X_test.csv'))
    #recordar cambiar ese X_test por el original que envio el profe!
    capa_oculta, capa_oculta2, capa_oculta3, capa_oculta4, output = red_neuronal.think(x_test)
    tocsv('./Y_result.csv', around(output).astype(float))
    #y_test_deseado = array(convertir_scsv('./Y_test_deseado.csv'))
    #tocsv('./Y_deseasoFlota.csv', around(y_test_deseado).astype(float))    
    calculatesucc(around(output).astype(float))

Entrenando
Fin entrenamiento
Probamos la red neuronal con datos de prueba: 
Porcentaje de aciertos
83.63%
